In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [58]:
table = pd.read_table('./1663769555_8559356_train.txt', names=['binding','sequence'])

In [107]:
from sklearn.feature_extraction.text import CountVectorizer
def return_binary_vectorized_form(data_frame):
    count_vectorizer = CountVectorizer(binary=True)
    fitted_and_transformed = count_vectorizer.fit_transform(data_frame).toarray()
    return count_vectorizer,  fitted_and_transformed

In [127]:
from sklearn.feature_extraction.text import CountVectorizer
def return_binary_vectorized_form_with_vocab(data_frame, vocab):
    count_vectorizer = CountVectorizer(binary=True, vocabulary=vocab)
    fitted_and_transformed = count_vectorizer.fit_transform(data_frame).toarray()
    return count_vectorizer,  fitted_and_transformed

In [60]:
from sklearn.feature_selection import SelectKBest, chi2
def k_best_features(feature_set, target_values, fraction, feature_list):
    select_bestselect_best = SelectKBest(score_func=chi2, k = int(len(feature_list)*fraction))
    select_bestselect_best.fit(feature_set,target_values)
    mask = select_bestselect_best.get_support()
    k_best_featurs = feature_list[mask]
    return k_best_featurs

In [61]:
def collect_with_target_value(data_frame, column, target_value):
    collected_values = data_frame.loc[data_frame[column] == target_value]
    return collected_values

In [62]:
def resample(data_frame, fraction):
    sampled_df = pd.DataFrame.sample(data_frame,frac=fraction, random_state= np.random.RandomState())
    return sampled_df

In [256]:
active_df = collect_with_target_value(table, 'binding', 1)
not_active_dfs = [resample(collect_with_target_value(table, 'binding',0), .105) for i in range(20)]

In [259]:
balanced_dfs = [pd.concat([active_df,not_active_dfs[i]]).sample(frac=1).reset_index(drop=True)  for i in range(20)]

In [264]:
balanced_and_transformed = [return_binary_vectorized_form(balanced_dfs[i]['sequence']) for i in range(20) ]
k_best_feature = [k_best_features(balanced_and_transformed[i][1],balanced_dfs[i]['binding'],.50,balanced_and_transformed[i][0].get_feature_names_out()) for i in range(20)]
# balanced_vectorizer, samples_balanced = return_binary_vectorized_form(balanced_df['sequence'])
# k_best_feature = k_best_features(samples_balanced,balanced_df['binding'],.50,balanced_vectorizer.get_feature_names_out())
# k_best_feature

[array(['100', '10007', '10008', ..., '9999', '99991', '99992'],
       dtype=object),
 array(['100', '10000', '10012', ..., '99994', '99997', '99999'],
       dtype=object),
 array(['10003', '10004', '10008', ..., '99997', '99998', '99999'],
       dtype=object),
 array(['10', '10000', '100000', ..., '99990', '99996', '99998'],
       dtype=object),
 array(['1000', '10000', '10003', ..., '99992', '99996', '99999'],
       dtype=object),
 array(['10000', '10005', '10006', ..., '99996', '99997', '99999'],
       dtype=object),
 array(['10002', '10004', '10005', ..., '99992', '99997', '99999'],
       dtype=object),
 array(['10', '10000', '10004', ..., '99988', '99989', '99991'],
       dtype=object),
 array(['10000', '10002', '10005', ..., '99991', '99992', '99998'],
       dtype=object),
 array(['10002', '10005', '10008', ..., '99994', '99996', '99999'],
       dtype=object),
 array(['100', '10002', '10003', ..., '99997', '99998', '99999'],
       dtype=object),
 array(['10002', '10007

In [265]:
from sklearn.model_selection import train_test_split
train_test = [ train_test_split(balanced_dfs[i]['sequence'],balanced_dfs[i]['binding'] ,train_size=.70, random_state=0) for i in range(20)]

In [282]:
train_features = [return_binary_vectorized_form_with_vocab(train_test[i][0],k_best_feature[i]) for i in range(20)]
test_transformed = [train_features[i][0].transform(train_test[i][1]) for i in range(20)]

In [284]:
# y_predict = bag_clf.predict(x_test_features)
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import  accuracy_score

bag_clf = [BaggingClassifier(DecisionTreeClassifier(criterion='gini'),n_estimators=500, max_samples=100,random_state=np.random.RandomState(), n_jobs=2) for i in range(20)]
fit_clf = [bag_clf[i].fit(train_features[i][1], train_test[i][2]) for i in range(20)]
predict_test = [ bag_clf[i].predict(test_transformed[i]) for i in range(20)]
accuracy_score = [ accuracy_score(predict_test[i], train_test[i][3]) for i in range(20)]

In [285]:
# for i in range(20):
#     for j in range(len(predict_test[i])):


[0.7446808510638298,
 0.7872340425531915,
 0.723404255319149,
 0.851063829787234,
 0.723404255319149,
 0.7659574468085106,
 0.8085106382978723,
 0.8297872340425532,
 0.8085106382978723,
 0.7659574468085106,
 0.7446808510638298,
 0.8297872340425532,
 0.8297872340425532,
 0.7872340425531915,
 0.8085106382978723,
 0.7659574468085106,
 0.8085106382978723,
 0.8297872340425532,
 0.7446808510638298,
 0.7446808510638298]

In [241]:
table_test = pd.read_table('./1663769555_8653905_test.txt', names=['sequence'])
test_features = vectorizer.transform(table_test['sequence']).toarray()

In [242]:
test_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [243]:
y_predict_unlabled = bag_clf.predict(test_features)

In [244]:
import csv
outfile = open('./results.csv','w')
out = csv.writer(outfile)
out.writerows(map(lambda x: [x], y_predict_unlabled))
outfile.close()